In [1]:
from urllib.request import urlopen as uReq #urllib will get the page itself
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import re
import nltk
import heapq
import requests
import sys
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import wikipedia
from selenium import webdriver
import os
import urllib.request
import urllib.parse
from tqdm import tqdm
import itertools

c:\Users\PSG3\anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [ ]:
therapeutic_justice_type = []

In [ ]:
website = 'https://www.judiciary.gov.sg/family'
user_agent = {'User-agent' : 'Mozilla/5.0'}
page = requests.get(website, headers=user_agent)
soup = BeautifulSoup(page.text, 'lxml')
fact_cards = soup.find_all('a', {'class': 'nav-link'})
tj_dict = {}
all_types = []

for fact_card in fact_cards[8:]:
    tj_type = fact_card.get_text().split('\n')[1]
    tj_desc = fact_card.get_text().split('\n')[2]
    tj_link = fact_card['href']
    tj_dict[tj_type] = [tj_desc, tj_link]
    all_types.append(tj_type)

tj_dict

In [124]:
desc_dict = {}
tdesc_dict = {}
ndesc_dict = {}
ddesc_dict = {}
link_dict = {}
row_dict = {}
nav_link_list = []

for type in all_types:
    row_link_list = []
    with open(type + ".txt", "w") as f:
        print(type)
        next_page = 'https://www.judiciary.gov.sg' + tj_dict[type][1]
        page = requests.get(next_page, headers=user_agent)
        soup = BeautifulSoup(page.text, 'lxml')
        content_desc = soup.find_all('div', {'class': 'sf-content-block'})
        accr_desc = soup.find_all('div', {'class': 'accordion-wrapper'})
        # This gets everything in a page, including the p and ul tags
        for desc in content_desc[2:-6]:
            desc_text = desc.get_text()
            f.write(desc_text.replace('\n', ' '))
            # print(desc_text.replace('\n', ' '))
            for link in desc.find_all('a'):
                desc_dict[link.string] = link['href']

        # This gets all the links in the accordion
        for desc in accr_desc:
            for link in desc.find_all('a'):
                link_dict[link.string] = link['href']
            
        if type =="Adoption" or type == "Protection for vulnerable adults" :
            timeline_desc = soup.find_all('div', {'class': 'timeline'})
            for t_desc in timeline_desc:
                timeline_text = t_desc.get_text()
                f.write(timeline_text.replace('\n', ' '))
                # print(timeline_text.replace('\n', ' '))
                for link in t_desc.find_all('a'):
                    tdesc_dict[link.string] = link['href']

        elif type == "Care and protection for children and young persons" or type == "Guardianship":
            nav_desc = soup.find_all('li', {'class': 'nav-item w-100'})
            for n_desc in nav_desc:
                # print(n_desc)
                nav_text = n_desc.get_text()
                f.write(nav_text.replace('\n', ' '))
                # print(nav_text.replace('\n', ' '))
                for link in n_desc.find_all('a'):
                    nav_link_list.append(link['href'])

        else:
            div_desc = soup.find_all('div', {'class': 'row'})
            if type != "Direct judicial communication for international family proceedings affecting children" and type != "Syariah Court orders":
                type_list = []
                for d_desc in div_desc[:-2]:
                    div_text = d_desc.get_text()
                    f.write(div_text.replace('\n', ' '))
                    # print(div_text.replace('\n', ' '))
                    type_list.append(list(filter(None,div_text.split("\n"))))
                    for link in d_desc.find_all('a'):
                        row_link_list.append(link['href'])
                    row_dict[type] = row_link_list

Adoption
Care and protection for children and young persons
Divorce
Deputyship
Direct judicial communication for international family proceedings affecting children
Family Justice Courts appeals
Family guidance for children and young persons
Guardianship
International child abduction
Probate and administration
Protection against family violence
Protection for vulnerable adults
Maintenance
Mediation and counselling in the Family Justice Courts
Syariah Court orders


In [132]:
def get_txt(txt_name, link):
    with open(txt_name, "a") as f:
        next_page = 'https://www.judiciary.gov.sg' + link
        page = requests.get(next_page, headers=user_agent)
        soup = BeautifulSoup(page.text, 'lxml')
        content_desc = soup.find_all('div', {'class': 'sf-content-block'})
        for desc in content_desc[2:-6]:
            desc_text = desc.get_text()
            f.write(desc_text)
            # for link in desc.find_all('a'):
            #     desc_dict[link.string] = link['href']

For Adoption

In [92]:
adoption_int_links = dict(itertools.islice(tdesc_dict.items(), 4))
for link in adoption_int_links.values():
    get_txt('adoption_internal.txt', link)

For Protection for vulnerable adults

In [93]:
protection_vulnerable_int_links =  dict(itertools.islice(tdesc_dict.items(), len(tdesc_dict)-2, None))
for link in protection_vulnerable_int_links.values():
    get_txt('protection_vulnerable_internal.txt', link)

For Care and protection for children and young persons

In [94]:
for link in nav_link_list[:2]:
    get_txt('care_protection_children_internal.txt', link)

Guardianship

In [95]:
for link in nav_link_list[2:]:
    get_txt('guardianship_internal.txt', link)

Divorce, Deputyship, Family justice courts appeals, Family guidance for children and young persons, International child abduction, Probate and administration, Protection against family violence, Maintenance, Mediation and counselling in the family justice courts

In [133]:
# iterate through the dictionary and get the key and value
for key, list_of_links in row_dict.items():
    key = key.replace(" ", "_").lower()
    for link in list_of_links:
        get_txt(key + '_internal.txt', link)

Civil Law

In [2]:
website = 'https://www.judiciary.gov.sg/civil'
user_agent = {'User-agent' : 'Mozilla/5.0'}
page = requests.get(website, headers=user_agent)
soup = BeautifulSoup(page.text, 'lxml')
fact_cards = soup.find_all('a', {'class': 'nav-link'})
tj_dict = {}
all_types = []

for fact_card in fact_cards[8:]:
    tj_type = fact_card.get_text().split('\n')[1]
    tj_desc = fact_card.get_text().split('\n')[2]
    tj_link = fact_card['href']
    tj_dict[tj_type] = [tj_desc, tj_link]
    all_types.append(tj_type)

tj_dict

{'Admiralty proceedings (from 1 April 2022)': ['Admiralty disputes involve ships or maritime activities. The High Court (General Division) can hear certain claims against ships, shipowners or ship charterers.',
  '/civil/admiralty-proceedings-(from-1-april-2022)'],
 'Appeals under the Protection from Online Falsehoods and Manipulation Act (POFMA) (from 1 April 2022)': ['Find out how to appeal to the High Court (General Division) against a direction or declaration under the Protection from Online Falsehoods and Manipulation Act.',
  '/civil/appeals-under-the-protection-from-online-falsehoods-and-manipulation-act-(pofma)-(from-1-april-2022)'],
 'Bankruptcy': ['Any individual or firm that is unable to repay any debt of at least $15,000 may be declared a bankrupt by the High Court (General Division). ',
  '/civil/bankruptcy'],
 'Bills of sale: registration and renewal (from 1 April 2022)': ['A bill of sale is a legal document that records the transfer of ownership of an asset. Register or 

In [8]:
desc_dict = {}
tdesc_dict = {}
ndesc_dict = {}
ddesc_dict = {}
link_dict = {}
row_dict = {}
nav_link_list = []

for type in all_types:
    row_link_list = []
    # with open(type + ".txt", "w") as f:
    print(type)
    next_page = 'https://www.judiciary.gov.sg' + tj_dict[type][1]
    page = requests.get(next_page, headers=user_agent)
    soup = BeautifulSoup(page.text, 'lxml')
    content_desc = soup.find_all('div', {'class': 'sf-content-block'})
    accr_desc = soup.find_all('div', {'class': 'accordion-wrapper'})
    # This gets everything in a page, including the p and ul tags
    for desc in content_desc:
        desc_text = desc.get_text()
        # f.write(desc_text.replace('\n', ' '))
        print(desc_text.replace('\n', ' '))
        for link in desc.find_all('a'):
            desc_dict[link.string] = link['href']

    # This gets all the links in the accordion
    for desc in accr_desc:
        for link in desc.find_all('a'):
            link_dict[link.string] = link['href']
    print('-------------------------------------------------------')
    
            

Admiralty proceedings (from 1 April 2022)
 A Singapore Government Agency Website   
 Who we areNews and resourcesLegal professionalsForms and feesLegal help and supportContact us 
  NoteThis page is for matters that the Rules of Court 2021 apply to. For content relating to matters that the Rules of Court 2014 apply, click here. If you are uncertain as to which version of the Rules of Court applies to your matter, click here.  
 What are admiralty proceedingsIn Singapore, admiralty law deals with certain claims against ships, shipowners or ship charterers that fall under the admiralty jurisdiction of the General Division of the High Court.Refer to Section 3 of the High Court (Admiralty Jurisdiction) Act for more information. Some examples of admiralty cases that the General Division of the High Court can hear include: Claims to the possession or ownership of a ship.Claims for damage done by a ship.Claims for damage received by a ship.Claims for loss of or damage to goods carried in a sh